In [0]:
%python
spark.conf.set(
    "your azure acount",
    "your key"
)

In [0]:
%python
# Parquetファイルを読み込む
df = spark.read.format("parquet").load("abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/source_files_001TB_parquet/catalog_returns")

# Deltaテーブルとして保存する
df.write.format("delta").mode("overwrite").save("abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_returns_delta")


In [0]:
SELECT * FROM delta.`abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_returns_delta`

In [0]:
OPTIMIZE delta.`abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_returns_delta`

In [0]:
-- Deltaテーブルのターゲットファイルサイズを512MBに設定
ALTER TABLE delta.`abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_returns_delta`
SET TBLPROPERTIES ('delta.targetFileSize' = '536870912');

-- OPTIMIZEコマンドを実行してファイルサイズを調整
OPTIMIZE delta.`abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_returns_delta`;


catalog sales

In [0]:
%python
df = spark.read.format("parquet").load("abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/source_files_001TB_parquet/catalog_sales")

df.write.format("delta").mode("overwrite").save("abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_sales_delta")

In [0]:
ALTER TABLE delta.`abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_sales_delta`
SET TBLPROPERTIES ('delta.targetFileSize' = '536870912');

OPTIMIZE delta.`abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_sales_delta`;

In [0]:
SHOW TBLPROPERTIES delta.`abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_sales_delta`;


In [0]:
DESCRIBE EXTENDED delta.`abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/delta/catalog_sales_delta`;


In [0]:
%python
# 必要なパッケージのインポート
from pyspark.sql import SparkSession

# Parquetファイルを読み込む
df = spark.read.format("parquet").load("abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/source_files_001TB_parquet/customer")

# データを再分割して書き込み
target_file_size_mib = 500  # 目標ファイルサイズ（MiB単位）

# Calculate the average bytes per row
total_size_bytes = df.rdd.map(lambda row: len(str(row))).sum()
total_rows = df.count()
avg_bytes_per_row = total_size_bytes / total_rows

rows_per_file = int((target_file_size_mib * 1024 * 1024) / avg_bytes_per_row)

df.repartition(rows_per_file).write \
  .format("parquet") \
  .mode("overwrite") \
  .save("abfss://datalake@adlsfabricperfomance.dfs.core.windows.net/raw/tpc-ds/source_files_001TB_parquet/customer")